In [5]:
%pip install -q Faker==25.8.0

Note: you may need to restart the kernel to use updated packages.


In [26]:
from pathlib import Path
import numpy as np
import pandas as pd

ROOT = Path(".").resolve()

for d in ["data/raw", "data/processed", "reports/figures", "sql", "presentation"]:
    (ROOT / d).mkdir(parents=True, exist_ok=True)

SEED = 42
rng = np.random.default_rng(SEED)
np.random.seed(SEED)

print("Estrutura criada em:", ROOT)
print("Versões -> pandas:", pd.__version__, "| numpy:", np.__version__)

Estrutura criada em: C:\DEV\Analise_de_vendas
Versões -> pandas: 2.3.2 | numpy: 2.2.6


In [28]:
from datetime import datetime, timedelta
from faker import Faker
import numpy as np
import pandas as pd

EXTENDER_ATE_2024_06 = False
N_LINHAS = 300
START_2023 = "2023-01-01"
END_2023 = "2023-12-31"
END_DATE = "2024-06-30" if EXTENDER_ATE_2024_06 else END_2023

CATEGORIAS = {
    "Eletrônicos": ["Fone Pro", "Mouse Óptico", "Teclado Mecânico", "Smartwatch", "Caixa Bluetooth"],
    "Casa & Cozinha": ["Liquidificador Turbo", "Panela Antiaderente", "Aspirador 900", "Jogo de Facas", "Cafeteira X"],
    "Esporte": ["Bola Oficial", "Tênis Running", "Garrafa Térmica", "Mochila Fitness", "Camiseta Dry"],
    "Moda": ["Camiseta Básica", "Calça Jeans", "Jaqueta Leve", "Tênis Casual", "Boné Snapback"],
    "Brinquedos": ["Quebra-cabeça 500pç", "Carrinho RC", "Boneca Classic", "Blocos Criativos", "Jogo de Tabuleiro"],
}

PRECO_PARAMS = {
    "Eletrônicos":  {"mu": np.log(300), "sigma": 0.6, "minv": 50},
    "Casa & Cozinha":{"mu": np.log(150), "sigma": 0.5, "minv": 30},
    "Esporte":      {"mu": np.log(120), "sigma": 0.5, "minv": 20},
    "Moda":         {"mu": np.log(90),  "sigma": 0.5, "minv": 20},
    "Brinquedos":   {"mu": np.log(80),  "sigma": 0.6, "minv": 15},
}

SAZONALIDADE = {
    1: 0.90, 2: 0.95, 3: 1.00, 4: 1.00, 5: 1.05, 6: 1.00,
    7: 1.10, 8: 1.05, 9: 1.00, 10: 1.10, 11: 1.20, 12: 1.30,
}

def gerar_datas(n, start_date: str, end_date: str, rng: np.random.Generator):
    start = datetime.fromisoformat(start_date)
    end = datetime.fromisoformat(end_date)
    delta_dias = (end - start).days + 1
    offs = rng.integers(0, delta_dias, size=n)
    return np.array([start + timedelta(days=int(o)) for o in offs])

def preco_por_categoria(cat: str, rng: np.random.Generator) -> float:
    p = PRECO_PARAMS[cat]
    val = rng.lognormal(mean=p["mu"], sigma=p["sigma"])
    val = max(val, p["minv"])
    return round(float(val), 2)

rng = np.random.default_rng(SEED)
fake = Faker("pt_BR")

n = max(N_LINHAS, 50)

categorias = rng.choice(list(CATEGORIAS.keys()), size=n)
produtos = np. array([rng.choice(CATEGORIAS[c]) for c in categorias])

datas = gerar_datas(n, START_2023, END_DATE, rng)

lambda_base = 3.0
fatores = np.array([SAZONALIDADE.get(d.month, 1.0) for d in datas])
quantidade = rng.poisson(lam=lambda_base * fatores, size=n) + 1

precos = np.array([preco_por_categoria(cat, rng) for cat in categorias])

ids = np.arange(1, n + 1)

df = pd.DataFrame({
    "ID": ids,
    "Data": pd.to_datetime(datas).strftime("%Y-%m-%d"),
    "Produto": produtos,
    "Categoria": categorias,
    "Quantidade": quantidade,
    "Preco": precos,
})


raw_path = ROOT / "data/raw/vendas_2023.csv"
raw_path.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(raw_path, index=False, encoding="utf-8")

print(f"CSV bruto salvo em: {raw_path}")
print("Linhas totais (inclui duplicatas intencionais):", len(df))
df.head()


CSV bruto salvo em: C:\DEV\Analise_de_vendas\data\raw\vendas_2023.csv
Linhas totais (inclui duplicatas intencionais): 300


,ID,Data,Produto,Categoria,Quantidade,Preco
0,1,2023-08-13,Mouse Óptico,Eletrônicos,1,275.60
1,2,2023-10-12,Boné Snapback,Moda,4,139.26
2,3,2023-09-27,Camiseta Básica,Moda,5,68.88
3,4,2023-02-19,Camiseta Dry,Esporte,4,54.67
4,5,2023-10-29,Garrafa Térmica,Esporte,5,60.13


In [29]:
raw_path = ROOT / "data/raw/vendas_2023.csv"
df = pd.read_csv(raw_path)

In [30]:
n = len(df)

if "rgn" not in globals():
    SEED = 42
    rng = np.random.default_rng(SEED)
r = rng.random

rates = {
    "categoria_nan":        0.15,  # 15% Categoria ausente
    "produto_nan":          0.07,  # 7%  Produto ausente
    "preco_nan":            0.12,  # 12% Preco ausente
    "quantidade_nan":       0.08,  # 8%  Quantidade ausente
    "quantidade_str":       0.12,  # 12% Quantidade vira string
    "quantidade_zero_neg":  0.04,  # 4%  Quantidade vira 0/-1 (inválido)
    "data_nan":             0.05,  # 5%  Data ausente
    "data_invalida":        0.04,  # 4%  Data inválida (strings que quebram o parser)
    "dup_linha":            0.05,  # 5%  duplicatas idênticas
    "dup_chave":            0.05,  # 5%  duplicatas por (ID,Data) com valores diferentes
}

m = r(n) < rates["categoria_nan"];  df.loc[m, "Categoria"] = None
m = r(n) < rates["produto_nan"];    df.loc[m, "Produto"] = None
m = r(n) < rates["preco_nan"];      df.loc[m, "Preco"] = None
m = r(n) < rates["quantidade_nan"]; df.loc[m, "Quantidade"] = None
m = r(n) < rates["data_nan"];       df.loc[m, "Data"] = None   

m = r(n) < rates["quantidade_str"]
df.loc[m, "Quantidade"] = (
    pd.to_numeric(df.loc[m, "Quantidade"], errors= "coerce")
      .fillna(0).astype(int).astype(str)
)

m = r(n) < rates["data_invalida"];  df.loc[m, "Data"] = "2023-13-01"
m = r(n) < rates["data_invalida"];  df.loc[m, "Data"] = "31/02/2023"

k = max(1, int(rates["dup_chave"] * n))
idx = rng.choice(df.index, size=k, replace=False)
dups = df.loc[idx].copy()

dups["Preco"] = pd.to_numeric(dups["Preco"], errors="coerce") * (1 + rng.normal(0, 0.05, size=len(dups)))
dups["Quantidade"] = (
    pd.to_numeric(dups["Quantidade"], errors="coerce").fillna(1).astype(int)
    + rng.integers(-1, 2, size=len(dups))
)

df = pd.concat([df, dups], ignore_index=True)
df = df.sort_values(by=['ID'], kind='mergesort').reset_index(drop=True)
df.to_csv(raw_path, index=False, encoding="utf-8")
print(f"[OK] Sujeiras aplicadas e salvo: {raw_path} | linhas={len(df)}")

[OK] Sujeiras aplicadas e salvo: C:\DEV\Analise_de_vendas\data\raw\vendas_2023.csv | linhas=315


C:\Users\crish\AppData\Local\Temp\ipykernel_12540\1094870035.py:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['0' '5' '3' '4' '2' '4' '4' '6' '3' '1' '5' '0' '4' '2' '5' '3' '9' '3'
 '4' '2' '2' '6' '10']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[m, "Quantidade"] = (


In [31]:
from pathlib import Path
import pandas as pd
import numpy as np
from IPython.display import display

def diagnosticar_csv_plus(
    path_csv,
    keys_opcionais=None,                 # ex.: [("ID","Data"), ("Produto","Data")]
    periodo=("2023-01-01","2024-06-30"), # janela válida para Data
    mostrar_head=3,
    sample_k=3                           # quantos exemplos de linhas problemáticas mostrar
):
    df = pd.read_csv(path_csv)
    nlin, ncol = df.shape
    print(f"Arquivo: {path_csv} | shape = ({nlin}, {ncol})\n")

    # 1) Head e dtypes
    if mostrar_head:
        print(f"Amostra (head {mostrar_head}):")
        display(df.head(mostrar_head))
    print("\nTipos por coluna:")
    display(df.dtypes.to_frame("dtype"))

    # 2) Nulos (contagem e %)
    print("\nValores ausentes por coluna:")
    nulos = df.isna().sum()
    pct   = (nulos / max(len(df),1) * 100).round(2)
    out_nulos = (pd.DataFrame({"nulos": nulos, "pct_%": pct})
                   .sort_values("nulos", ascending=False))
    display(out_nulos)

    # 3) Duplicatas (linha inteira + por chaves)
    print("\nDuplicatas (linha inteira):", int(df.duplicated().sum()))
    if keys_opcionais:
        rows = []
        # normaliza: aceitar ("ID","Data") ou [("ID","Data"), ...]
        if isinstance(keys_opcionais, (tuple, list)) and all(isinstance(x, str) for x in keys_opcionais):
            keys_opcionais = [tuple(keys_opcionais)]
        for ch in keys_opcionais:
            ch = tuple(ch)
            rows.append({"chaves": str(ch),
                         "duplicatas": int(df.duplicated(subset=list(ch)).sum())})
        print("\nDuplicatas por chave:")
        display(pd.DataFrame(rows).sort_values("duplicatas", ascending=False, ignore_index=True))

    # 4) Texto que parece número (tipo misto)
    print("\nColunas 'object' que parecem numéricas:")
    rows = []
    for col in df.columns:
        if df[col].dtype == "object":
            conv = pd.to_numeric(df[col], errors="coerce")
            mask = df[col].notna() & conv.notna()
            qtd  = int(mask.sum())
            if qtd > 0:
                exemplos = (df.loc[mask, col].astype(str).drop_duplicates()[:sample_k].tolist())
                rows.append({"coluna": col, "qtd_texto_numerico": qtd, "pct_%": round(qtd/len(df)*100,2),
                             "exemplos": exemplos})
    if rows:
        display(pd.DataFrame(rows).sort_values("qtd_texto_numerico", ascending=False, ignore_index=True))
    else:
        print("  (nenhuma)")

    # 5) Problemas específicos por coluna

    # 5.1 Data: inválida e fora do período
    if "Data" in df.columns:
        dt = pd.to_datetime(df["Data"], errors="coerce")
        mask_invalida = dt.isna() & df["Data"].notna()     # strings inválidas (não conta NaN já acima)
        inicio, fim = pd.to_datetime(periodo[0]), pd.to_datetime(periodo[1])
        mask_fora = dt.notna() & ((dt < inicio) | (dt > fim))
        print("\n[Data] inválida:", int(mask_invalida.sum()), " | fora do período:", int(mask_fora.sum()))
        if mask_invalida.any():
            print("Exemplos Data inválida:")
            display(df.loc[mask_invalida].head(sample_k))
        if mask_fora.any():
            print("Exemplos Data fora do período:")
            display(df.loc[mask_fora].head(sample_k))

    # 5.2 Quantidade: não numérica, zeros/negativos
    if "Quantidade" in df.columns:
        q_conv = pd.to_numeric(df["Quantidade"], errors="coerce")
        mask_nao_num = df["Quantidade"].notna() & q_conv.isna()
        mask_zero_neg = q_conv.notna() & (q_conv <= 0)
        print("\n[Quantidade] não numérica:", int(mask_nao_num.sum()), " | <= 0:", int(mask_zero_neg.sum()))
        if mask_nao_num.any():
            print("Exemplos Quantidade não numérica:")
            display(df.loc[mask_nao_num].head(sample_k))
        if mask_zero_neg.any():
            print("Exemplos Quantidade <= 0:")
            display(df.loc[mask_zero_neg].head(sample_k))

    # 5.3 Preco: não numérico, <= 0
    if "Preco" in df.columns:
        p_conv = pd.to_numeric(df["Preco"], errors="coerce")
        mask_nao_num = df["Preco"].notna() & p_conv.isna()
        mask_le_zero = p_conv.notna() & (p_conv <= 0)
        print("\n[Preco] não numérico:", int(mask_nao_num.sum()), " | <= 0:", int(mask_le_zero.sum()))
        if mask_nao_num.any():
            print("Exemplos Preco não numérico:")
            display(df.loc[mask_nao_num].head(sample_k))
        if mask_le_zero.any():
            print("Exemplos Preco <= 0:")
            display(df.loc[mask_le_zero].head(sample_k))

    return df  # permite encadear outras análises

resultado = diagnosticar_csv_plus(raw_path, keys_opcionais=[("ID","Data")])
resultado.head()



Arquivo: C:\DEV\Analise_de_vendas\data\raw\vendas_2023.csv | shape = (315, 6)

Amostra (head 3):


,ID,Data,Produto,Categoria,Quantidade,Preco
0,1,2023-08-13,Mouse Óptico,Eletrônicos,1.0,275.60
1,2,2023-10-12,Boné Snapback,Moda,4.0,139.26
2,3,2023-09-27,Camiseta Básica,Moda,NaN,68.88



Tipos por coluna:


,dtype
ID,int64
Data,object
Produto,object
Categoria,object
Quantidade,float64
Preco,float64



Valores ausentes por coluna:


...

,ID,Data,Produto,Categoria,Quantidade,Preco
14,15,31/02/2023,Calça Jeans,Moda,3.0,96.59
17,18,31/02/2023,Smartwatch,NaN,4.0,939.67
21,22,31/02/2023,Liquidificador Turbo,Casa & Cozinha,3.0,138.19



[Quantidade] não numérica: 0  | <= 0: 2
Exemplos Quantidade <= 0:


,ID,Data,Produto,Categoria,Quantidade,Preco
16,17,2023-04-02,Mochila Fitness,Esporte,0.0,135.09
175,169,2023-02-21,Liquidificador Turbo,Casa & Cozinha,0.0,NaN



[Preco] não numérico: 0  | <= 0: 0


,ID,Data,Produto,Categoria,Quantidade,Preco
0,1,2023-08-13,Mouse Óptico,Eletrônicos,1.0,275.60
1,2,2023-10-12,Boné Snapback,Moda,4.0,139.26
2,3,2023-09-27,Camiseta Básica,Moda,NaN,68.88
3,4,2023-02-19,Camiseta Dry,Esporte,4.0,54.67
4,5,NaN,Garrafa Térmica,NaN,5.0,60.13


In [33]:
from pathlib import Path
from typing import Dict, Tuple, Iterable, Optional
import numpy as np
import pandas as pd

# ---------------------------
# Configuração básica
# ---------------------------
COLS = ["ID", "Data", "Produto", "Categoria", "Quantidade", "Preco"]

# Defina a raiz do projeto: aqui uso a pasta do próprio script
ROOT = Path(".").resolve()
 # ajuste se preferir

RAW_PATH   = (ROOT / "data" / "raw" / "vendas_2023.csv").resolve()
CLEAN_PATH = (ROOT / "data" / "processed" / "data_clean.csv").resolve()

print("RAW_PATH  :", RAW_PATH)
print("CLEAN_PATH:", CLEAN_PATH)

# ---------------------------
# Helpers
# ---------------------------
def _first_non_null(s: pd.Series):
    s2 = s.dropna()
    return s2.iloc[0] if not s2.empty else pd.NA

def _mode_or_first(s: pd.Series):
    s2 = s.dropna()
    if s2.empty:
        return pd.NA
    m = s2.mode()
    return m.iloc[0] if not m.empty else s2.iloc[0]

def _to_num(s: pd.Series):
    return pd.to_numeric(s, errors="coerce")

def _parse_dates_robusto(s: pd.Series) -> pd.Series:
    """
    2-pass: 1) ISO (YYYY-MM-DD) com yearfirst  2) dd/mm/yyyy com dayfirst
    Evita a ambiguidade de dayfirst com ISO.
    """
    # Passo 1: tentar ISO/ano-primeiro
    parsed1 = pd.to_datetime(s, errors="coerce", yearfirst=True)
    # Passo 2: preencher os NaT restantes assumindo dd/mm/yyyy
    mask = parsed1.isna()
    if mask.any():
        parsed2 = pd.to_datetime(s[mask], errors="coerce", dayfirst=True)
        parsed1.loc[mask] = parsed2
    return parsed1

# ---------------------------
# Função principal
# ---------------------------
def clean_sales_csv(input_path, output_path=None, key_cols=("ID", "Data")) -> Tuple[pd.DataFrame, Dict[str, int]]:
    input_path = Path(input_path)

    if output_path is None:
        output_path = input_path.with_name(f"{input_path.stem}_clean.csv")
    else:
        output_path = Path(output_path)

    # Garante que a pasta de saída existe
    output_path.parent.mkdir(parents=True, exist_ok=True)

    # Leitura
    df = pd.read_csv(input_path)
    df = df.copy()  # trabalhar numa cópia

    # Verificação de colunas
    missing = [c for c in COLS if c not in df.columns]
    if missing:
        raise ValueError(f"Colunas ausentes no arquivo: {missing}")
    df = df[COLS]

    report: Dict[str, int] = {}

    # 1) Remoção de duplicatas idênticas (linha inteira)
    dup_full = int(df.duplicated().sum())
    df = df.drop_duplicates(keep="first").reset_index(drop=True)
    report["duplicatas_linha_removidas"] = dup_full

    # 2) Normalização de strings
    for c in ["Produto", "Categoria"]:
        df[c] = df[c].astype("string").str.strip()

    # 3) Datas (robusto a ISO e dd/mm/yyyy)
    parsed = _parse_dates_robusto(df["Data"])
    invalid_dates = int(parsed.isna().sum())
    df["Data"] = parsed
    # descartamos datas inválidas
    df = df[df["Data"].notna()].copy()
    report["linhas_descartadas_data_invalida"] = invalid_dates

    # 4) Quantidade e Preço -> numérico
    df["Quantidade"] = _to_num(df["Quantidade"])
    df["Preco"] = _to_num(df["Preco"])

    # 5) Flags de valores inválidos (<=0) -> NaN
    q_nonpos = int((df["Quantidade"] <= 0).fillna(False).sum())
    p_nonpos = int((df["Preco"] <= 0).fillna(False).sum())
    df.loc[df["Quantidade"] <= 0, "Quantidade"] = np.nan
    df.loc[df["Preco"] <= 0, "Preco"] = np.nan
    report["quantidade_nao_positiva_para_NaN"] = q_nonpos
    report["preco_nao_positivo_para_NaN"] = p_nonpos

    # 6) Imputações por Produto (medianas) + fallback global
    med_preco_prod = df.groupby("Produto", dropna=False)["Preco"].median()
    med_qtd_prod = df.groupby("Produto", dropna=False)["Quantidade"].median()

    # Preço
    p_missing_before = int(df["Preco"].isna().sum())
    df["Preco"] = df["Preco"].fillna(df["Produto"].map(med_preco_prod))
    df["Preco"] = df["Preco"].fillna(df["Preco"].median())
    report["preco_imputado"] = p_missing_before - int(df["Preco"].isna().sum())

    # Quantidade
    q_missing_before = int(df["Quantidade"].isna().sum())
    df["Quantidade"] = df["Quantidade"].fillna(df["Produto"].map(med_qtd_prod))
    df["Quantidade"] = df["Quantidade"].fillna(1)
    report["quantidade_imputada"] = q_missing_before - int(df["Quantidade"].isna().sum())

    # 7) Duplicatas por chave (ID, Data): consolidar
    dup_key_count = int(df.duplicated(subset=list(key_cols)).sum())
    report["duplicatas_por_chave_encontradas"] = dup_key_count

    def _agg_quantidade(s: pd.Series):
        v = _to_num(s)
        return float(np.nanmedian(v)) if v.notna().any() else np.nan

    def _agg_preco(s: pd.Series):
        v = _to_num(s)
        return float(np.nanmedian(v)) if v.notna().any() else np.nan

    df = (
        df.groupby(list(key_cols), as_index=False)
          .agg({
              "Produto": _mode_or_first,
              "Categoria": _mode_or_first,
              "Quantidade": _agg_quantidade,
              "Preco": _agg_preco
          })
    )

    # 8) Recupera Produto/Categoria pelos mapeamentos mais frequentes
    prod_missing_before = int(df['Produto'].isna().sum())
    cat_missing_before = int(df['Categoria'].isna().sum())
    prod_to_cat = (
        df[['Produto', 'Categoria']]
          .dropna()
          .drop_duplicates()
          .groupby('Produto')
          ['Categoria']
          .agg(lambda s: s.mode().iat[0])
    )
    df['Categoria'] = df['Categoria'].fillna(df['Produto'].map(prod_to_cat))
    cat_to_prod = (
        df[['Categoria', 'Produto']]
          .dropna()
          .drop_duplicates()
          .groupby('Categoria')
          ['Produto']
          .agg(lambda s: s.mode().iat[0])
    )
    df['Produto'] = df['Produto'].fillna(df['Categoria'].map(cat_to_prod))
    prod_missing_after = int(df['Produto'].isna().sum())
    cat_missing_after = int(df['Categoria'].isna().sum())
    report['produto_imputado'] = prod_missing_before - prod_missing_after
    report['categoria_imputada'] = cat_missing_before - cat_missing_after
    mask_drop = df['Produto'].isna() | df['Categoria'].isna()
    if mask_drop.any():
        report['linhas_descartadas_sem_produto_ou_categoria'] = int(mask_drop.sum())
        df = df.loc[~mask_drop].copy()

    # 9) Tipos finais e ajustes
    df["ID"] = df["ID"].astype("int64")
    df["Quantidade"] = pd.Series(df["Quantidade"]).round().astype("Int64")  # inteiro anulável
    df["Preco"] = pd.Series(df["Preco"]).astype("float64")
    df["Produto"] = df["Produto"].astype("string")
    df["Categoria"] = df["Categoria"].astype("string")

    # 10) Salvar LIMPO (agora sim)
    df = df.sort_values(by=list(key_cols), kind='mergesort').reset_index(drop=True)
    df.to_csv(output_path, index=False, encoding="utf-8")

    # 11) Métricas finais
    report["linhas_finais"] = int(len(df))
    report["arquivo_limpo"] = str(output_path)

    return df, report


# ===== Execução =====
if __name__ == "__main__":
    df_clean, resumo = clean_sales_csv(RAW_PATH, output_path=CLEAN_PATH, key_cols=("ID", "Data"))

    # Validação rápida (lendo do disco)
    df_disk = pd.read_csv(CLEAN_PATH)
    print("len(df_disk) (lido do disco):", len(df_disk))

    print("Resumo da limpeza:")
    for k, v in resumo.items():
        print(f"- {k}: {v}")

    # Espiar dtypes
    print(df_clean.dtypes)


RAW_PATH  : C:\DEV\Analise_de_vendas\data\raw\vendas_2023.csv
CLEAN_PATH: C:\DEV\Analise_de_vendas\data\processed\data_clean.csv


C:\Users\crish\AppData\Local\Temp\ipykernel_12540\599978275.py:48: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  parsed2 = pd.to_datetime(s[mask], errors="coerce", dayfirst=True)


len(df_disk) (lido do disco): 271
Resumo da limpeza:
- duplicatas_linha_removidas: 0
- linhas_descartadas_data_invalida: 30
- quantidade_nao_positiva_para_NaN: 2
- preco_nao_positivo_para_NaN: 0
- preco_imputado: 40
- quantidade_imputada: 16
- duplicatas_por_chave_encontradas: 13
- produto_imputado: 13
- categoria_imputada: 45
- linhas_descartadas_sem_produto_ou_categoria: 1
- linhas_finais: 271
- arquivo_limpo: C:\DEV\Analise_de_vendas\data\processed\data_clean.csv
ID                     int64
Data          datetime64[ns]
Produto       string[python]
Categoria     string[python]
Quantidade             Int64
Preco                float64
dtype: object
